<a href="https://colab.research.google.com/github/aikokass82-design/home-tasks-/blob/main/Diplom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Дипломный проект**

***часть 1***

In [ ]:
import pandas as pd
import sqlalchemy
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')
engine = sqlalchemy.create_engine("sqlite+pysqlite:////content/drive/MyDrive/Colab Notebooks/sales_land_auctions.db", echo=False)

In [ ]:
query = sqlalchemy.text("""
SELECT *
FROM auctions
""")
table_part = pd.read_sql(query, con=engine)
table_part.head(5)

In [ ]:
query = sqlalchemy.text("""
SELECT *
FROM classificator
""")
table_part = pd.read_sql(query, con=engine)
table_part.head(5)

In [ ]:
query = sqlalchemy.text("""
SELECT *
FROM additional_classificator
""")
table_part = pd.read_sql(query, con=engine)
table_part.head(5)

In [ ]:
query = sqlalchemy.text("""
SELECT *
FROM customers
""")
table_part = pd.read_sql(query, con=engine)
table_part.head(5)

***часть 2***

In [ ]:
auctions = pd.read_sql("SELECT * FROM auctions", engine)
customers = pd.read_sql("SELECT * FROM customers", engine)
classificator = pd.read_sql("SELECT * FROM classificator", engine)
additional_classificator = pd.read_sql("SELECT * FROM additional_classificator", engine)

df = auctions.merge(customers, how='left', on='customer_edrpou')

df = df.merge(classificator, how='left', left_on='classificator', right_on='classificator_code')

df = df.merge(additional_classificator, how='left', left_on='additional_classificator', right_on='code')

df

In [ ]:
#1
query = sqlalchemy.text("""
SELECT oblast,
SUM(land_area) as total_area
FROM auctions
GROUP BY oblast
ORDER BY total_area DESC""")
sum_area = pd.read_sql(query, engine)
sum_area

In [ ]:
plt.figure()
sum_area.plot(kind='bar')
plt.title("Суммарная площадь по областям")
plt.xlabel("Область")
plt.ylabel("Площадь (га)")
plt.tight_layout()
plt.show()

In [ ]:
#2
query= sqlalchemy.text("""
SELECT
oblast,
COUNT(*) AS lot_count
FROM auctions
GROUP BY oblast
ORDER BY lot_count DESC
""")

lots_count= pd.read_sql(query, con=engine)
print(lots_count)

In [ ]:
plt.figure()
lots_count.plot(kind='bar')
plt.title("Количество лотов по областям")
plt.xlabel("Область")
plt.ylabel("Количество")
plt.tight_layout()
plt.show()

In [ ]:
#3
query = sqlalchemy.text("""
    SELECT
        strftime('%Y', auction_date) AS year,
        oblast,
        COUNT(*) AS lot_count
    FROM auctions
    GROUP BY year, oblast
    ORDER BY year DESC, lot_count DESC
""")

lots_year = pd.read_sql(query, con=engine)
print(lots_year)

In [ ]:
import seaborn as sns
lots_year['lot_count'] = pd.to_numeric(lots_year['lot_count'], errors='coerce')
lots_year['year'] = lots_year['year'].astype(str)
pivot = lots_year.pivot(index='oblast', columns='year', values='lot_count')

plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, fmt="g", cmap="Blues")
plt.title("Количество лотов по годам по областям")
plt.xlabel("Год")
plt.ylabel("Область")
plt.tight_layout()
plt.show()

In [ ]:
#4
query = sqlalchemy.text("""
    SELECT
        a.oblast,
        c.classificator_code,
        c.classificator_description,
        COUNT(*) AS lot_count
    FROM auctions a
    JOIN classificator c
        ON a.classificator = c.classificator_code
    GROUP BY a.oblast, c.classificator_code, c.classificator_description
""")

df = pd.read_sql(query, con=engine)

most_common_per_oblast = df.loc[df.groupby('oblast')['lot_count'].idxmax()]

print(most_common_per_oblast)

In [ ]:
#5
industrial_lands = df[df['classificator_description'].str.contains('промислов', case=False, na=False)]
count_by_oblast = industrial_lands['oblast'].value_counts()
top_oblast = count_by_oblast.idxmax()
max_count = count_by_oblast.max()
print(f"Больше всего земель под промышленную застройку выставлено в области: {top_oblast} ({max_count} участков)")

In [ ]:
#6
lot_counts = df.groupby(['oblast', 'classificator_description']).size().reset_index(name='lot_count')
lot_counts = lot_counts.sort_values(by='lot_count', ascending=False)
print(lot_counts)

***часть 3***

In [ ]:
auctions = pd.read_sql("SELECT * FROM auctions", engine)
print(auctions.columns)
df = auctions.merge(customers, how='left', on='customer_edrpou')
df = df.merge(classificator, how='left', left_on='classificator', right_on='classificator_code')
df = df.merge(additional_classificator, how='left', left_on='additional_classificator', right_on='code')
print(df.columns)
df['auction_date'] = pd.to_datetime(df['auction_date'])


In [ ]:
import requests
df['auction_date'] = pd.to_datetime(df['auction_date'])
min_date = df['auction_date'].min()
max_date = df['auction_date'].max()
unique_dates = df['auction_date'].dt.strftime('%Y%m%d').unique()

all_rates = []
for date_str in unique_dates:
    url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={date_str}&json"
    response = requests.get(url)
    data = response.json()
    all_rates.extend(data)

eur_rates = pd.DataFrame(all_rates)
eur_rates['exchangedate'] = pd.to_datetime(eur_rates['exchangedate'])
df = df.merge(eur_rates[['exchangedate', 'rate']],
              left_on='auction_date', right_on='exchangedate',
              how='left')
print(df[['auction_date', 'oblast', 'classificator', 'rate']].head())


In [ ]:
import requests
df["auction_date"] = pd.to_datetime(df["auction_date"])

min_date = df["auction_date"].min()
max_date = df["auction_date"].max()

print("Минимальная дата:", min_date)
print("Максимальная дата:", max_date)

start = min_date.strftime("%Y%m%d")
end = max_date.strftime("%Y%m%d")

url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={date_str}&json"

response = requests.get(url)
data = response.json()

eur_rates = pd.DataFrame(data)
eur_rates["exchangedate"] = pd.to_datetime(eur_rates["exchangedate"])

print(eur_rates.head())

In [ ]:
import requests
df["auction_date"] = pd.to_datetime(df["auction_date"])

min_date = df["auction_date"].min()
max_date = df["auction_date"].max()

start = min_date.strftime("%Y%m%d")
end = max_date.strftime("%Y%m%d")

eur_url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={start}&json"
response = requests.get(eur_url)
data = response.json()
eur_rates = pd.DataFrame(data)
eur_rates["exchangedate"] = pd.to_datetime(eur_rates["exchangedate"])
unique_dates = df["auction_date"].dt.strftime("%Y%m%d").unique()

all_rates = []
for date_str in unique_dates:
    url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=EUR&date={date_str}&json"
    response = requests.get(url)
    data = response.json()
    all_rates.extend(data)

eur_rates = pd.DataFrame(all_rates)
eur_rates["exchangedate"] = pd.to_datetime(eur_rates["exchangedate"])

df = df.merge(eur_rates[["exchangedate", "rate"]],
              left_on="auction_date", right_on="exchangedate",
              how="left")

print(df[["auction_date", "oblast", "classificator_description", "rate"]].head())


In [ ]:
print(df.columns)